## import necessary csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
events_italy = pd.read_csv("srcd_data/events_Italy.csv")
events_italy.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,tagsList,pos_orig_y,pos_orig_x,pos_dest_y,pos_dest_x
0,8,Simple pass,[{'id': 1801}],8327,"[{'y': 52, 'x': 49}, {'y': 44, 'x': 43}]",2575959,Pass,3158,1H,2.530536,85.0,180423957,[1801],52,49,44,43
1,8,Simple pass,[{'id': 1801}],20438,"[{'y': 44, 'x': 43}, {'y': 17, 'x': 36}]",2575959,Pass,3158,1H,3.768418,85.0,180423958,[1801],44,43,17,36
2,7,Touch,[],8306,"[{'y': 17, 'x': 36}, {'y': 56, 'x': 78}]",2575959,Others on the ball,3158,1H,4.868265,72.0,180423959,NaN,17,36,56,78
3,1,Ground attacking duel,"[{'id': 504}, {'id': 703}, {'id': 1801}]",8306,"[{'y': 56, 'x': 78}, {'y': 15, 'x': 64}]",2575959,Duel,3158,1H,8.114676,11.0,180423960,"[504, 703, 1801]",56,78,15,64
4,1,Ground attacking duel,"[{'id': 503}, {'id': 703}, {'id': 1801}]",8306,"[{'y': 15, 'x': 64}, {'y': 15, 'x': 72}]",2575959,Duel,3158,1H,8.647892,11.0,180423961,"[503, 703, 1801]",15,64,15,72


## filter necessary columns

In [3]:
italy_shots = events_italy[events_italy["eventName"] == "Shot"]
italy_shots = italy_shots[['eventId', 'subEventName', 'matchId', 'teamId', 'matchPeriod', 'eventSec']]
italy_shots.head()

,eventId,subEventName,matchId,teamId,matchPeriod,eventSec
68,10,Shot,2575959,3172,1H,193.924279
90,10,Shot,2575959,3158,1H,250.843152
213,10,Shot,2575959,3158,1H,633.577774
426,10,Shot,2575959,3158,1H,1359.469422
435,10,Shot,2575959,3172,1H,1412.601525


In [4]:
## adjust all instances to pure minute 
italy_shots.loc[italy_shots["matchPeriod"] == "2H", "eventSec"] += (45 * 60)

In [5]:
italy_shots = italy_shots[['eventId', 'subEventName', 'matchId', 'teamId', 'eventSec']]
italy_shots["eventMin"] = round(italy_shots["eventSec"] / 60, 0)
italy_shots.head()

,eventId,subEventName,matchId,teamId,eventSec,eventMin
68,10,Shot,2575959,3172,193.924279,3.0
90,10,Shot,2575959,3158,250.843152,4.0
213,10,Shot,2575959,3158,633.577774,11.0
426,10,Shot,2575959,3158,1359.469422,23.0
435,10,Shot,2575959,3172,1412.601525,24.0


## group shots by game

In [6]:
shots_ind_by_game = dict((italy_shots.groupby("matchId").apply(lambda x: list(dict(x["eventMin"]).values()), include_groups=False)))

In [7]:
dInfo = {"matchId": ["filler"], "shots": [["fller"]]}
italy_sbm = pd.DataFrame(dInfo) 
italy_sbm = italy_sbm.drop(index=0) 
italy_sbm["matchId"] = shots_ind_by_game.keys()
italy_sbm["shots"] = shots_ind_by_game.values()


In [8]:
shots_per_game = [] 
late_game_shots_by_game = [] 
for row in italy_sbm.iterrows():
    game_shots = len(row[1]["shots"])
    lg_shots = len(list(filter(lambda x: x >= 60, row[1]["shots"])))

    late_game_shots_by_game.append(lg_shots)
    shots_per_game.append(game_shots)

In [9]:
italy_sbm["total shots"] = shots_per_game
italy_sbm["late game shots"] = late_game_shots_by_game
italy_sbm["late game shot density"] =  italy_sbm["late game shots"] / italy_sbm["total shots"]

In [10]:
italy_sbm

,matchId,shots,total shots,late game shots,late game shot density
0,2575959,"[3.0, 4.0, 11.0, 23.0, 24.0, 24.0, 25.0, 28.0,...",22,9,0.409091
1,2575960,"[2.0, 4.0, 8.0, 12.0, 16.0, 18.0, 25.0, 27.0, ...",19,4,0.210526
2,2575961,"[13.0, 15.0, 17.0, 18.0, 20.0, 22.0, 23.0, 31....",22,9,0.409091
3,2575962,"[3.0, 17.0, 19.0, 22.0, 24.0, 26.0, 28.0, 37.0...",29,10,0.344828
4,2575963,"[9.0, 11.0, 11.0, 14.0, 16.0, 18.0, 23.0, 25.0...",27,10,0.370370
...,...,...,...,...,...
375,2576334,"[1.0, 4.0, 13.0, 15.0, 23.0, 24.0, 25.0, 26.0,...",30,10,0.333333
376,2576335,"[5.0, 5.0, 6.0, 7.0, 8.0, 9.0, 15.0, 21.0, 28....",28,7,0.250000
377,2576336,"[4.0, 9.0, 12.0, 12.0, 12.0, 17.0, 18.0, 22.0,...",29,9,0.310345
378,2576337,"[1.0, 2.0, 18.0, 18.0, 31.0, 40.0, 41.0, 43.0,...",23,10,0.434783


## relevant df's 

In [11]:
print(italy_shots)

        eventId subEventName  matchId  teamId     eventSec  eventMin
68           10         Shot  2575959    3172   193.924279       3.0
90           10         Shot  2575959    3158   250.843152       4.0
213          10         Shot  2575959    3158   633.577774      11.0
426          10         Shot  2575959    3158  1359.469422      23.0
435          10         Shot  2575959    3172  1412.601525      24.0
...         ...          ...      ...     ...          ...       ...
646870       10         Shot  2576338    3193  3852.032980      64.0
646904       10         Shot  2576338    3193  3951.730517      66.0
647169       10         Shot  2576338    3193  4765.034482      79.0
647218       10         Shot  2576338    3193  5067.252041      84.0
647286       10         Shot  2576338    3193  5279.867806      88.0

[8806 rows x 6 columns]


In [12]:
print(italy_sbm)

     matchId                                              shots  total shots  \
0    2575959  [3.0, 4.0, 11.0, 23.0, 24.0, 24.0, 25.0, 28.0,...           22   
1    2575960  [2.0, 4.0, 8.0, 12.0, 16.0, 18.0, 25.0, 27.0, ...           19   
2    2575961  [13.0, 15.0, 17.0, 18.0, 20.0, 22.0, 23.0, 31....           22   
3    2575962  [3.0, 17.0, 19.0, 22.0, 24.0, 26.0, 28.0, 37.0...           29   
4    2575963  [9.0, 11.0, 11.0, 14.0, 16.0, 18.0, 23.0, 25.0...           27   
..       ...                                                ...          ...   
375  2576334  [1.0, 4.0, 13.0, 15.0, 23.0, 24.0, 25.0, 26.0,...           30   
376  2576335  [5.0, 5.0, 6.0, 7.0, 8.0, 9.0, 15.0, 21.0, 28....           28   
377  2576336  [4.0, 9.0, 12.0, 12.0, 12.0, 17.0, 18.0, 22.0,...           29   
378  2576337  [1.0, 2.0, 18.0, 18.0, 31.0, 40.0, 41.0, 43.0,...           23   
379  2576338  [16.0, 17.0, 30.0, 44.0, 49.0, 53.0, 53.0, 56....           15   

     late game shots  late game shot de

In [18]:
italy_shots.to_csv("prepped_data/italy_shots.csv")

In [14]:
italy_sbm.to_csv("prepped_data/italy_sbm.csv")